<a href="https://colab.research.google.com/github/mloyorev/Theory/blob/main/4_InventoryDynamics.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import numpy as np
import matplotlib.pyplot as plt
from collections import namedtuple
from numba import njit, int32

# **Solving the inventory model**

We solve the inventory model **using the value function iterion (VFI) algorithm**.


We define a function that returns the index of the maximum element on a list

In [5]:
@njit # The function is compiled using numba's Just-In-Time compiler

def argmax(list_object):
  max_val = -np.inf    # Initialize the maximum value
  argmax_index = None  # Initialize the index of the maximum value
  for i, x in enumerate(list_object): # Initialize a loop for each pair (i,x) in enumerate(list_object)
    if x > max_val:      # If an (i,x) satisfies that x > max_ val, then: - max_val = x
      max_val = x        #                                                - arg_max = i
      argmax_index = i
  return argmax_index

We also use the succesive aproximation function, as defined on the previous notebook.

In [4]:
def succesive_approx(T,                # Operator
                     x_0,              # Initial condition
                     tol=1e-6,         # Error tolerance
                     max_iter=10_000,  # Maximum number of iteration
                     print_step=25,    # Frequency with which progress messages will be printed during iterations
                     verbose=False):   # Will not print progress messages during iterations

  x = x_0           # Initialize: - Initial condition
  error = tol + 1   #             - Error
  k = 1             #             - Number of iterations

  while error > tol and k <= max_iter:    # Initialize a bucle that keeps iterating while error > tol and k <= max_iter
    x_new = T(x)                          # Apply transformation
    error = np.max(np.abs(x_new - x))     # Gets the error with the supremum norm

    if verbose and k % print_step == 0:                      # If verbose=True and k is a multiple of print_step, progress message is printed
      print(f"Completed iteration {k} with error {error}.")

    x = x_new    # Update: - Initial condition
    k +=1        #         - Number of iterations

  if error > tol:
    print(f"Warning: Iteration hit upper bound {max_iter}.")
  elif verbose:
    print(f"Terminanted succesufully in {k} iterations.")

  return x

To store our model parameters, we use a namedtuple. Here we define the namedtuple and a function to create instances with default values.

In [7]:
Model = namedtuple('Model', ('beta',    # Discount factor
                             'kappa',   # Maximum inventory capacity
                             'c',       # Unit cost of stock
                             'K',       # Fixed cost of ordering stock
                             'p',       # Demand probability mass function parameter
                             'phi',     # Demanda probability mass function
                             'Gamma'))  # Feasible space for stock

def create_inventory_model(r=0.02,   # Interest rate
                           kappa=40, # Maximum inventory
                           c=0.2,    # Cost parameter
                           K=2,      # Cost parameter
                           p=0.6):   # Demand parameter

  # Define the discount factor
  beta = 1 / (1+r)

  # Define a geometric distribution on d = 0, 1, 2, ... with parameter p
  @njit
  def phi(d):
    return (1 - p) ** d * p

  # Define a feasible correspondence of space at the inventory
  @njit
  def Gamma(x):
    return range(kappa - x + 1)

  return Model(beta=beta, kappa=kappa, c=c, K=K, p=p, phi=phi, Gamma=Gamma)

The next function is the unmaximized RHS of the Bellman equation, which is:

$$B(x,a,v):=r(x,a)+\beta∑_{d}v[F(x,a,d)]\phi(d)=\text{profit}+\beta\text{continuation value}$$

In [8]:
@njit
def B(x, a, v, model, d_max=100):
  beta, kappa, c, K, p, phi, Gamma = model  # Unpacked model parameters

  expected_revenue   = sum([min(x,d) * phi(d) for d in range(d_max)])
  profit             = expected_revenue - c * a - K * (a > 0)
  continuation_value = sum([v[max(x - d, 0) + a] * phi(d) for d in range(d_max)])

  return profit + beta * continuation_value
